# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,taber,49.8167,-112.1518,-9.99,89,100,1.03,CA,1707157952
1,1,al jawf,29.5000,38.7500,11.21,37,0,1.85,SA,1707157952
2,2,adamstown,-25.0660,-130.1015,23.53,85,100,7.45,PN,1707157952
3,3,college,64.8569,-147.8028,-23.08,74,20,0.00,US,1707157952
4,4,grytviken,-54.2811,-36.5092,8.38,64,64,5.63,GS,1707157952


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng",
                                      "Lat",
                                      geo = True,
                                      tiles = "OSM",
                                      size = "Humidity",
                                      color = "City",
                                      xlim = [-180, 180],
                                      ylim = [-60, 80],
                                      xlabel = "Longitude",
                                      ylabel = "Latitude")

# The following source described customization parameters:
# https://hvplot.holoviz.org/user_guide/Customization.html
                                      
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_filter_1_df = city_data_df.loc[city_data_df["Max Temp"] < 27]
city_data_filter_2_df = city_data_filter_1_df.loc[city_data_filter_1_df["Max Temp"] > 21]
city_data_filter_3_df = city_data_filter_2_df.loc[city_data_filter_2_df["Wind Speed"] < 4.5]
city_data_filter_4_df = city_data_filter_3_df.loc[city_data_filter_3_df["Cloudiness"] == 0]

# Drop any rows with null values
city_data_filter_5_df = city_data_filter_4_df.dropna()

# Display sample data
city_data_filter_5_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,kampong thom,12.7111,104.8887,26.76,61,0,2.55,KH,1707157956
27,27,ixtapa,20.7000,-105.2000,24.57,47,0,1.54,MX,1707157957
86,86,bredasdorp,-34.5322,20.0403,22.62,71,0,3.40,ZA,1707157966
173,173,east london,-33.0153,27.9116,23.52,83,0,2.06,ZA,1707157985
199,199,sarina,-21.4333,149.2167,23.67,84,0,1.64,AU,1707157990
207,207,sawang daen din,17.4753,103.4575,23.95,29,0,1.53,TH,1707157991
214,214,saint-pierre,-21.3393,55.4781,23.82,69,0,4.12,RE,1707157992
226,226,allapalli,19.4167,80.0667,23.68,50,0,1.60,IN,1707157994
244,244,hermanus,-34.4187,19.2345,21.56,83,0,3.82,ZA,1707157997
248,248,el naranjo,21.4667,-98.6000,26.10,33,0,1.74,MX,1707157998


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_filter_5_df.iloc[:, [1, 8, 2, 3, 5]].copy()

# The source below gave a good explanation for and example for use of .copy():
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.copy.html

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# The source below provided exampled for how to add an empty column to a dataframe:
# https://stackoverflow.com/questions/16327055/how-to-add-an-empty-column-to-a-dataframe

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
15,kampong thom,KH,12.7111,104.8887,61,
27,ixtapa,MX,20.7000,-105.2000,47,
86,bredasdorp,ZA,-34.5322,20.0403,71,
173,east london,ZA,-33.0153,27.9116,83,
199,sarina,AU,-21.4333,149.2167,84,
207,sawang daen din,TH,17.4753,103.4575,29,
214,saint-pierre,RE,-21.3393,55.4781,69,
226,allapalli,IN,19.4167,80.0667,50,
244,hermanus,ZA,-34.4187,19.2345,83,
248,el naranjo,MX,21.4667,-98.6000,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": f"accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kampong thom - nearest hotel: សណ្ឋាគារអរុណរះ
ixtapa - nearest hotel: Ma. Cristina
bredasdorp - nearest hotel: Victoria Hotel
east london - nearest hotel: No hotel found
sarina - nearest hotel: Tandara Hotel
sawang daen din - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
allapalli - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
el naranjo - nearest hotel: No hotel found
fochville - nearest hotel: No hotel found
annigeri - nearest hotel: No hotel found
polasara - nearest hotel: No hotel found
quelimane - nearest hotel: Mabassa
murud - nearest hotel: No hotel found
kadod - nearest hotel: No hotel found
cova figueira - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
15,kampong thom,KH,12.7111,104.8887,61,សណ្ឋាគារអរុណរះ
27,ixtapa,MX,20.7000,-105.2000,47,Ma. Cristina
86,bredasdorp,ZA,-34.5322,20.0403,71,Victoria Hotel
173,east london,ZA,-33.0153,27.9116,83,No hotel found
199,sarina,AU,-21.4333,149.2167,84,Tandara Hotel
207,sawang daen din,TH,17.4753,103.4575,29,No hotel found
214,saint-pierre,RE,-21.3393,55.4781,69,Tropic Hotel
226,allapalli,IN,19.4167,80.0667,50,No hotel found
244,hermanus,ZA,-34.4187,19.2345,83,Aloe guest house
248,el naranjo,MX,21.4667,-98.6000,33,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points("Lng",
                                    "Lat",
                                    geo = True,
                                    tiles = "OSM",
                                    size = "Humidity",
                                    color = "City",
                                    xlim = [-180, 180],
                                    ylim = [-60, 80],
                                    xlabel = "Longitude",
                                    ylabel = "Latitude",
                                    hover_cols = ["Hotel Name", "Country"])

# Again, the site below provided description of the customization parameters:
# https://hvplot.holoviz.org/user_guide/Customization.html
                                      
# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)